In [1]:
from approx_aligned_decoding.backtracking_strategy.fast_gad_backtracking import FastGADBacktrackingStrategy
from approx_aligned_decoding.backtracking_strategy.grammar_aligned_decoding_backtracking import \
    GrammarAlignedDecodingBacktrackingStrategy
from approx_aligned_decoding.backtracking_strategy.naive_backtracking import NaiveBacktrackingStrategy
from approx_aligned_decoding.hallucination_detector.combined_hallucination_detector import \
    CombinedHallucinationDetector
from approx_aligned_decoding.stopping_criteria.stop_words import StopWords
%load_ext autoreload
from approx_aligned_decoding.utils import set_seed
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from approx_aligned_decoding.hallucination_detector.banned_text_hallucination_detector import \
    BannedTextHallucinationDetector
import torch


In [3]:
from approx_aligned_decoding.generator import generate
from transformers import BitsAndBytesConfig

In [4]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16)

In [5]:
device_main = "cuda:0"

In [6]:
model = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model, quantization_config=quant_config, device_map=device_main)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
banned_letter = "U"
input = f"""<s>[INST]Talk about the value of interactive examples without using the letter "{banned_letter}".[/INST]"""
hallu_detector = CombinedHallucinationDetector(BannedTextHallucinationDetector(banned_letter.upper()), BannedTextHallucinationDetector(banned_letter.lower()))
inputs = tokenizer(input, return_tensors="pt").to(model.device)

In [8]:
backtracking_strategy = FastGADBacktrackingStrategy()  # Our method
#backtracking_strategy = GrammarAlignedDecodingBacktrackingStrategy()  # ASAp
#backtracking_strategy = NaiveBacktrackingStrategy()  # Constrained generation- might also need to set move_hallu_idx_forward for more complex detectors than BannedText

In [9]:
print(tokenizer.decode(model.generate(**inputs, max_new_tokens=200, do_sample=True)[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s>[INST]Talk about the value of interactive examples without using the letter "U".[/INST] Interactive examples are valuable learning tools that engage learners in active participation and hands-on experience. They offer the opportunity to explore concepts in real-time, allowing learners to experiment, observe outcomes, and adjust actions based on results. This immediate feedback mechanism aids in understanding complex ideas and reinforcing newly acquired skills. Interactive examples also cater to diverse learning styles and foster a more personalized and effective learning experience compared to traditional lectures or passive reading.</s>


In [10]:
for i in range(3):
    torch.cuda.empty_cache()
    result, stats = generate(model, max_new_tokens=200, max_total_invocations=2000,
                 hallucination_detector=hallu_detector,
                 left_context="", right_context="", tokenizer=tokenizer,
                 temperature=0.8, backtracking_strategy=backtracking_strategy,
                 top_k=20,
                 seed_func=lambda x: set_seed((i, x)),
                 stopping_criteria=StopWords(["</s>"]),
                 print_probability_trace_info=False,
                 **inputs)
    print(input + tokenizer.decode(result))
    print(stats)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


<s>[INST]Talk about the value of interactive examples without using the letter "U".[/INST]Interactive examples are an essential learning tool that significantly enhances the teaching and learning process. They provide a platform for learners to engage directly with concepts, principles, or ideas, enabling them to practice and apply the knowledge in a hands-on manner.

Interactive examples promote a more dynamic and personalized learning experience. Instead of merely reading or watching passively, learners can actively participate and receive instant feedback. This immediate feedback helps learners identify areas of strength and weaknesses, allowing them to refine their skills and knowledge more efficiently.

Interactive examples can also cater to different learning styles, as they can be designed to incorporate a range of sensory and interactive elements, from images and graphics to animated demonstrations and problem-solving activities. This diversity caters to learners with varied le